In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

### Preparación de datasets

In [2]:
from sklearn.preprocessing import LabelEncoder

def dataset_prep(window=3, neutral=True):
    
    train = pd.read_csv('data/trainset.csv', index_col=0)
    test  = pd.read_csv('data/testset.csv', index_col=0)
    
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    
    for col in cols1:
        train[col+'w'] = train[col].rolling(window=window).sum()
        test[col+'w'] = test[col].rolling(window=window).sum()
    
    train['exc_ret'] = train['exc_ret'].shift(periods=-1)
    test['exc_ret']  = test['exc_ret'].shift(periods=-1)
    
    train.dropna(axis=0, inplace=True)
    test.dropna(axis=0, inplace=True)
    
    cols2 = [x+'w' for x in cols1]
    
    X = train[cols2]
    y = train.exc_ret
    Xt = test[cols2]
    yt = test.exc_ret
    
    le = LabelEncoder()
    y  = le.fit_transform(y)
    yt = le.fit_transform(yt)
    
    return X, Xt, y, yt    

### Entrenamiento de Clasificadores

In [3]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_roc

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier 

In [4]:
def train_classifier(X, Xt, y, yt):
    
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  LogisticRegression(),
                  CatBoostClassifier()]
    params1 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 25),     # Maximum tree depth for base learners.
    }
    
    params2 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 25),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    params4 = {}
    
    model_params = [params1, params2, params2, params3, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    for name, mod, params in zip(model_name, model_init, model_params):
        model = mod
        kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=kf,
                                   scoring='roc_auc', n_iter=50, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results.sort_values(by='Test Score', ascending=False)

In [5]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = dataset_prep(window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   21.5s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


...

177:	learn: 0.6089838	total: 1.24s	remaining: 5.72s
178:	learn: 0.6084489	total: 1.25s	remaining: 5.75s
179:	learn: 0.6080535	total: 1.26s	remaining: 5.74s
180:	learn: 0.6076788	total: 1.27s	remaining: 5.73s
181:	learn: 0.6076180	total: 1.27s	remaining: 5.72s
182:	learn: 0.6074353	total: 1.28s	remaining: 5.71s
183:	learn: 0.6072283	total: 1.28s	remaining: 5.69s
184:	learn: 0.6068551	total: 1.29s	remaining: 5.68s
185:	learn: 0.6065543	total: 1.29s	remaining: 5.67s
186:	learn: 0.6061929	total: 1.3s	remaining: 5.66s
187:	learn: 0.6058243	total: 1.31s	remaining: 5.64s
188:	learn: 0.6057102	total: 1.31s	remaining: 5.63s
189:	learn: 0.6054856	total: 1.32s	remaining: 5.62s
190:	learn: 0.6053241	total: 1.32s	remaining: 5.61s
191:	learn: 0.6048601	total: 1.33s	remaining: 5.59s
192:	learn: 0.6045333	total: 1.33s	remaining: 5.58s
193:	learn: 0.6042802	total: 1.34s	remaining: 5.57s
194:	learn: 0.6041574	total: 1.35s	remaining: 5.57s
195:	learn: 0.6039986	total: 1.36s	remaining: 5.57s
196:	learn: 0

355:	learn: 0.5701646	total: 2.4s	remaining: 4.33s
356:	learn: 0.5696443	total: 2.41s	remaining: 4.34s
357:	learn: 0.5694930	total: 2.42s	remaining: 4.34s
358:	learn: 0.5694154	total: 2.44s	remaining: 4.35s
359:	learn: 0.5691861	total: 2.44s	remaining: 4.34s
360:	learn: 0.5689658	total: 2.45s	remaining: 4.33s
361:	learn: 0.5687440	total: 2.45s	remaining: 4.33s
362:	learn: 0.5685552	total: 2.46s	remaining: 4.32s
363:	learn: 0.5684436	total: 2.47s	remaining: 4.31s
364:	learn: 0.5684020	total: 2.47s	remaining: 4.3s
365:	learn: 0.5680585	total: 2.48s	remaining: 4.29s
366:	learn: 0.5679216	total: 2.48s	remaining: 4.28s
367:	learn: 0.5677795	total: 2.49s	remaining: 4.27s
368:	learn: 0.5676375	total: 2.49s	remaining: 4.26s
369:	learn: 0.5673914	total: 2.5s	remaining: 4.26s
370:	learn: 0.5672237	total: 2.5s	remaining: 4.25s
371:	learn: 0.5670862	total: 2.51s	remaining: 4.24s
372:	learn: 0.5669866	total: 2.52s	remaining: 4.23s
373:	learn: 0.5668113	total: 2.52s	remaining: 4.22s
374:	learn: 0.56

538:	learn: 0.5406609	total: 3.56s	remaining: 3.04s
539:	learn: 0.5404882	total: 3.57s	remaining: 3.04s
540:	learn: 0.5400841	total: 3.58s	remaining: 3.04s
541:	learn: 0.5399530	total: 3.58s	remaining: 3.03s
542:	learn: 0.5398404	total: 3.59s	remaining: 3.02s
543:	learn: 0.5397816	total: 3.6s	remaining: 3.02s
544:	learn: 0.5394108	total: 3.6s	remaining: 3.01s
545:	learn: 0.5394062	total: 3.61s	remaining: 3s
546:	learn: 0.5393424	total: 3.61s	remaining: 2.99s
547:	learn: 0.5392722	total: 3.62s	remaining: 2.98s
548:	learn: 0.5392417	total: 3.63s	remaining: 2.98s
549:	learn: 0.5391857	total: 3.63s	remaining: 2.97s
550:	learn: 0.5390855	total: 3.64s	remaining: 2.96s
551:	learn: 0.5389125	total: 3.64s	remaining: 2.96s
552:	learn: 0.5388860	total: 3.65s	remaining: 2.95s
553:	learn: 0.5388830	total: 3.65s	remaining: 2.94s
554:	learn: 0.5387963	total: 3.66s	remaining: 2.93s
555:	learn: 0.5386112	total: 3.66s	remaining: 2.92s
556:	learn: 0.5381924	total: 3.67s	remaining: 2.92s
557:	learn: 0.537

700:	learn: 0.5197929	total: 4.54s	remaining: 1.94s
701:	learn: 0.5197476	total: 4.55s	remaining: 1.93s
702:	learn: 0.5195883	total: 4.56s	remaining: 1.93s
703:	learn: 0.5195484	total: 4.57s	remaining: 1.92s
704:	learn: 0.5193897	total: 4.58s	remaining: 1.92s
705:	learn: 0.5190944	total: 4.58s	remaining: 1.91s
706:	learn: 0.5188626	total: 4.59s	remaining: 1.9s
707:	learn: 0.5185792	total: 4.6s	remaining: 1.9s
708:	learn: 0.5184758	total: 4.6s	remaining: 1.89s
709:	learn: 0.5180557	total: 4.61s	remaining: 1.88s
710:	learn: 0.5180273	total: 4.61s	remaining: 1.88s
711:	learn: 0.5175628	total: 4.62s	remaining: 1.87s
712:	learn: 0.5174038	total: 4.62s	remaining: 1.86s
713:	learn: 0.5171795	total: 4.63s	remaining: 1.85s
714:	learn: 0.5171336	total: 4.64s	remaining: 1.85s
715:	learn: 0.5168748	total: 4.64s	remaining: 1.84s
716:	learn: 0.5168102	total: 4.65s	remaining: 1.83s
717:	learn: 0.5167695	total: 4.65s	remaining: 1.83s
718:	learn: 0.5164790	total: 4.66s	remaining: 1.82s
719:	learn: 0.51

889:	learn: 0.4912259	total: 5.71s	remaining: 706ms
890:	learn: 0.4911357	total: 5.72s	remaining: 700ms
891:	learn: 0.4908806	total: 5.73s	remaining: 694ms
892:	learn: 0.4908355	total: 5.74s	remaining: 687ms
893:	learn: 0.4907977	total: 5.74s	remaining: 681ms
894:	learn: 0.4907414	total: 5.75s	remaining: 674ms
895:	learn: 0.4906850	total: 5.75s	remaining: 668ms
896:	learn: 0.4906126	total: 5.76s	remaining: 661ms
897:	learn: 0.4905474	total: 5.76s	remaining: 655ms
898:	learn: 0.4904957	total: 5.77s	remaining: 648ms
899:	learn: 0.4904323	total: 5.78s	remaining: 642ms
900:	learn: 0.4903436	total: 5.78s	remaining: 635ms
901:	learn: 0.4902538	total: 5.79s	remaining: 629ms
902:	learn: 0.4901866	total: 5.79s	remaining: 622ms
903:	learn: 0.4899372	total: 5.8s	remaining: 616ms
904:	learn: 0.4896281	total: 5.8s	remaining: 609ms
905:	learn: 0.4895656	total: 5.81s	remaining: 603ms
906:	learn: 0.4895120	total: 5.82s	remaining: 596ms
907:	learn: 0.4894178	total: 5.82s	remaining: 590ms
908:	learn: 0.

In [7]:
resultado.sort_values(by='Train Score', ascending=False).iloc[:, :-1].to_csv('class-vader.csv')